<a href="https://colab.research.google.com/github/marcelomdea/uninter/blob/main/marcelo_uninter_big_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Prática 01 – Somatório de IDs

In [14]:
from pyspark.sql import SparkSession

spark_RU4100225 = SparkSession.builder.appName("SomaIdFilmesNegativos").getOrCreate()
df = spark_RU4100225.read.csv("imdb-reviews-pt-br.csv", header=True, inferSchema=True)
filmes_negativos = df.filter(df["sentiment"] == "neg")
soma_ids = filmes_negativos.agg({"id": "sum"})
soma_ids.show()
print("Marcelo Moreira de Almeida - RU 4100225")

+---------+
|  sum(id)|
+---------+
|247015948|
+---------+

Marcelo Moreira de Almeida - RU 4100225


Prática 02 – Diferença do número de palavras totais de português para inglês os textos negativos

In [20]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import split, explode, lower, regexp_replace, size, col, sum

# Inicializar a sessão Spark
spark_RU4100225 = SparkSession.builder.appName("ComparacaoPalavras").getOrCreate()

# Carregar o DataFrame
df = spark_RU4100225.read.csv("imdb-reviews-pt-br.csv", header=True, inferSchema=True)

# Filtrar avaliações negativas
neg_reviews = df.filter(df["sentiment"] == "neg")

# Função para contar palavras em uma string
def contar_palavras(texto):
    palavras = texto.lower().split()
    return len(palavras)

# Registrar a função como uma UDF (User Defined Function) do Spark
spark_RU4100225.udf.register("contar_palavras_udf", contar_palavras)


# Contar palavras em português
port_reviews = neg_reviews.filter(df["text_pt"].isNotNull())
port_word_count = port_reviews.withColumn("word_count_pt", size(split(lower(regexp_replace("text_pt", "[^a-zA-ZÀ-ÿ ]", "")), " "))) \
    .agg(sum("word_count_pt").alias("total_words_pt"))

# Contar palavras em inglês
eng_reviews = neg_reviews.filter(df["text_en"].isNotNull())
eng_word_count = eng_reviews.withColumn("word_count_en", size(split(lower(regexp_replace("text_en", "[^a-zA-Z ]", "")), " "))) \
    .agg(sum("word_count_en").alias("total_words_en"))

# Calcular a diferença
print(port_word_count.collect()[0]["total_words_pt"])
print(eng_word_count.collect()[0]["total_words_en"])
diff = port_word_count.collect()[0]["total_words_pt"] - eng_word_count.collect()[0]["total_words_en"]

# Exibir o resultado
print(f"Diferença de palavras: {diff}")

spark_RU4100225.stop()
print("Marcelo Moreira de Almeida - RU 4100225")

2415179
2399590
Diferença de palavras: 15589
Marcelo Moreira de Almeida - RU 4100225
